### Mô tả bài toán
Bộ dữ liệu subset-covid-data.csv chứa thông tin về tình hình covid ngày 12/4/2020 Hãy thực hiện những kiểm định sau với mức ý nghĩa 5%:

1. Có mối tương quan nào giữa dân số và số ca nhiễm bệnh hay không
2. Có mối tương quan nào giữa số ca mắc và ca tử vong ở các quốc gia hay không
3. Hãy tiến hành rời rạc hóa thuộc tính population thành các nhóm theo thứ tự tăng dần đặt tên là biến population_ordinal:
- 1: nếu population < tứ phân vị thứ nhất
- 2: Population nằm trong khoảng từ tứ phân vị thứ nhất tới tứ phân vị thứ 2
- 3: nếu population nằm trong khoảng từ tứ phân vị thứ 2 tới thứ 3
- 4: nếu population >= tứ phân vị thứ 3
Hãy kiểm tra mối tương quan giữa thuộc tính population_ordinaly với số ca nhiễm bệnh

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [3]:
data = pd.read_csv('subset-covid-data.csv')
data.head(3)

,country,continent,date,day,month,year,cases,deaths,country_code,population
0,Afghanistan,Asia,2020-04-12,12,4,2020,34,3,AFG,37172386.0
1,Albania,Europe,2020-04-12,12,4,2020,17,0,ALB,2866376.0
2,Algeria,Africa,2020-04-12,12,4,2020,64,19,DZA,42228429.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       206 non-null    object 
 1   continent     206 non-null    object 
 2   date          206 non-null    object 
 3   day           206 non-null    int64  
 4   month         206 non-null    int64  
 5   year          206 non-null    int64  
 6   cases         206 non-null    int64  
 7   deaths        206 non-null    int64  
 8   country_code  202 non-null    object 
 9   population    202 non-null    float64
dtypes: float64(1), int64(5), object(4)
memory usage: 16.2+ KB


In [5]:
data.describe()

,day,month,year,cases,deaths,population
count,206.000000,206.000000,206.0,206.000000,206.000000,2.020000e+02
mean,12.024272,3.985437,2020.0,395.247573,29.713592,3.731716e+07
std,0.424141,0.209020,0.0,2160.176439,159.533131,1.413776e+08
min,11.000000,1.000000,2020.0,0.000000,0.000000,1.000000e+03
25%,12.000000,4.000000,2020.0,0.000000,0.000000,1.278222e+06
50%,12.000000,4.000000,2020.0,11.500000,0.000000,7.042862e+06
75%,12.000000,4.000000,2020.0,88.750000,3.000000,2.600084e+07
max,18.000000,4.000000,2020.0,28391.000000,1831.000000,1.392730e+09


#### 1. Có mối tương quan nào giữa dân số và số ca nhiễm bệnh hay không

Với mức ý nghĩa 5%: <br>
    H0: Giữa dân số và số ca nhiễm bệnh không có mối tương quan <br>
    H1: Giữa dân số và số ca nhiễm bệnh có mối tương quan

Vì đây là hai biến định lượng nên ta sẽ sử dụng kiểm định pearsonr

In [6]:
data_1 = data.filter(['cases', 'population'])
data_1.dropna(inplace=True)

In [7]:
r, pvalue = pearsonr(data_1['population'], data_1['cases'])
print('r = ', r)
print('pvalue = ', pvalue)

r =  0.1755392659429401
pvalue =  0.01246031070420873


Do p_value<0.05, bác bỏ H0, kết luận Giữa biến dân số và số ca nhiễm bệnh có mối tương quan

#### 2. Có mối tương quan nào giữa số ca mắc và ca tử vong ở các quốc gia hay không

Với mức ý nghĩa 5%: <br>
    H0: Giữa số ca mắc và ca tử vong không có mối tương quan <br>
    H1: Giữa số ca mắc và ca tử vong có mối tương quan

In [8]:
r, pvalue = pearsonr(data['cases'], data['deaths'])
print('r = ', r)
print('pvalue = ', pvalue)

r =  0.9432201374746727
pvalue =  1.3451948745846463e-99


pvalue <0.05 nên bác bỏ H0, chấp nhận H1, kết luận giữa ca mắc và số ca tử vong có tương quan tuyến tính mạnh với nhau

#### 3. Hãy tiến hành rời rạc hóa thuộc tính population thành các nhóm theo thứ tự tăng dần đặt tên là biến population_ordinal:
- 1: nếu population < tứ phân vị thứ nhất
- 2: Population nằm trong khoảng từ tứ phân vị thứ nhất tới tứ phân vị thứ 2
- 3: nếu population nằm trong khoảng từ tứ phân vị thứ 2 tới thứ 3
- 4: nếu population >= tứ phân vị thứ 3

In [9]:
q1,q2,q3 = data_1['population'].quantile(0.25), data_1['population'].quantile(0.5), data_1['population'].quantile(0.75)
def population_order(population): ##một cách ordinal_encoding
    if population <q1:
        return 1
    elif population>=q1 and population <q2:
        return 2
    elif population>=q2 and population <q3:
        return 3
    else: 
        return 4
data_1['population_ordinal'] = data_1['population'].apply(population_order)
data_1.head()

,cases,population,population_ordinal
0,34,37172386.0,4
1,17,2866376.0,2
2,64,42228429.0,4
3,21,77006.0,1
4,0,30809762.0,4


Do population_ordinal là dữ liệu kiểu thứ bậc nên sử dụng kiểm định spearman để kiểm định tương quan giữa population_ordinal và cases

In [10]:
from scipy import stats
r,pvalue = stats.spearmanr(data_1['population_ordinal'], data_1['cases'])
print ("r: ", r, "; pvalue: ", pvalue)

r:  0.49498466493711596 ; pvalue:  6.982786622314858e-14


Pvalue gần bằng 0 <0.05, và r=0.49 chứng tỏ giữa thuộc tính population_ordinal và số ca nhiễm bệnh có tương quan với nhau. Như vậy có thể thấy, việc biến đổi dữ liệu thành các khoảng có thể làm nổi bật thông tin tương quan giữa các thuộc tính.